In [51]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML
from matplotlib import animation

### Punto fermo

In [63]:
# create a figure and axes
fig = plt.figure(figsize=(5,5))
ax = plt.subplot()

# set up the subplots as needed
ax.set_xlim((0, 10))
ax.set_ylim((0, 10))
ax.set_xlabel('x')
ax.set_ylabel('y')

# create objects that will change in the animation. These are
# initially empty, and will be given new values for each frame
# in the animation.
pt, = ax.plot([], [], 'b.', ms=20)     # ax.plot returns a list of 2D line objects
plt.close()

In [64]:
# animation function. This is called sequentially
def disegnaframe(t):
    x0 = 5
    y0 = 5
    x = x0
    y = y0
    
    pt.set_data(x, y)
    ax.set_title('Tempo = {0:4d} s'.format(t))

In [65]:
anim = animation.FuncAnimation(fig, disegnaframe, frames=100, interval=20)

In [66]:
HTML(anim.to_html5_video())

### Punto in moto uniforme

In [ ]:
# create a figure and axes
fig = plt.figure(figsize=(5,5))
ax = plt.subplot()

# set up the subplots as needed
ax.set_xlim((0, 10))
ax.set_ylim((0, 10))
ax.set_xlabel('x')
ax.set_ylabel('y')

# create objects that will change in the animation. These are
# initially empty, and will be given new values for each frame
# in the animation.
pt, = ax.plot([], [], 'b.', ms=20)     # ax.plot returns a list of 2D line objects
plt.close()

In [ ]:
def disegnaframe(t):
    x0 = 5
    y0 = 5
    vx = 0.05
    vy = 0
    
    x = x0 + t * vx
    y = y0 + t * vy
    
    pt.set_data(x, y)
    ax.set_title('Tempo = {0:4d} s'.format(t))

In [ ]:
anim = animation.FuncAnimation(fig, disegnaframe, frames=100, interval=20)

In [ ]:
HTML(anim.to_html5_video())

### Punto in moto accelerato, forza costante

## <font color='red'>caduta gravi, moto proiettili</font>  

In [ ]:
# create a figure and axes
fig = plt.figure(figsize=(5,5))
ax = plt.subplot()

# set up the subplots as needed
ax.set_xlim((0, 10))
ax.set_ylim((0, 10))
ax.set_xlabel('x')
ax.set_ylabel('y')

# create objects that will change in the animation. These are
# initially empty, and will be given new values for each frame
# in the animation.
pt, = ax.plot([], [], 'b.', ms=20)     # ax.plot returns a list of 2D line objects
plt.close()

In [ ]:
def disegnaframe(t):
    m = 5  #kg
    Fx = 0.01 #N
    Fy = 0 #N
    x0 = 0 #m
    y0 = 5 #m
    v0x = 0.01 #m/s
    v0y = 0    #m/s
    
    ax = Fx/m
    ay = Fy/m
    x = x0 + v0x * t + 0.5 * ax * t**2
    y = y0 + v0y * t + 0.5 * ay * t**2
    
    pt.set_data(x, y)
    
    ax.set_title('Tempo = {0:4d} s'.format(t))

In [ ]:
anim = animation.FuncAnimation(fig, disegnaframe, frames=100, interval=20)

In [ ]:
HTML(anim.to_html5_video())

### Due punti, forza gravitazionale

In [ ]:
# create a figure and axes
fig = plt.figure(figsize=(5,5))
ax = plt.subplot()

# set up the subplots as needed
ax.set_xlim((0, 10))
ax.set_ylim((0, 10))
ax.set_xlabel('x')
ax.set_ylabel('y')

# create objects that will change in the animation. These are
# initially empty, and will be given new values for each frame
# in the animation.
pt1, = ax.plot([], [], 'b.', ms=20)     # ax.plot returns a list of 2D line objects
pt2, = ax.plot([], [], 'b.', ms=20)     # ax.plot returns a list of 2D line objects
plt.close()

## <font color='red'>mettere formula Fgrav, aggiungere conservazione energia, alcuni algoritmi non la conservano</font>  

In [ ]:
def getAcc(pos):
    """
    Calculate the acceleration on each particle due to Newton's Law 
    pos  is an N x 3 matrix of positions
    mass is an N x 1 vector of masses
    G is Newton's Gravitational constant
    softening is the softening length
    a is N x 3 matrix of accelerations
    """
    
    N = pos.shape[0]
    a = np.zeros((N,2))
    softening = 0.1 # softening length
    G = 1.0         # Newton's Gravitational Constant
    global mass

    for i in range(N):
        for j in range(N):
            dx = pos[j,0] - pos[i,0];
            dy = pos[j,1] - pos[i,1];
            inv_r3 = (dx**2 + dy**2 + softening**2)**(-1.5);
            a[i,0] +=  G * (dx * inv_r3) * mass[j];
            a[i,1] +=  G * (dy * inv_r3) * mass[j];
            
    return a

# Simulation parameters
N  = 2    # Number of particles
dt = 0.01 # timestep

# Generate Initial Conditions
#np.random.seed(42) # set the random number generator seed
mass = 20.0*np.ones((N,1))/N  # total mass of particles is 20
pos  = np.random.normal(5,2,(N,2))#np.random.randn(N,2)+5 # randomly selected positions and velocities
vel  = np.random.normal(0,1,(N,2))#np.random.randn(N,2)

# calculate initial gravitational accelerations
acc = getAcc(pos)

def disegnaframe(t):
    global pos
    global vel
    global acc
    global dt
    
    # (1/2) kick
    vel += acc * dt/2.0

    # drift
    pos += vel * dt

    # update accelerations
    acc = getAcc(pos)

    # (1/2) kick
    vel += acc * dt/2.0

    pt1.set_data(pos[0,0], pos[0,1])
    pt2.set_data(pos[1,0], pos[1,1])

    ax.set_title('Tempo = {0:4d} s'.format(t))

In [ ]:
anim = animation.FuncAnimation(fig, disegnaframe, frames=100, interval=20)

In [ ]:
HTML(anim.to_html5_video())

### N punti, forza gravitazionale

In [ ]:
# create a figure and axes
fig = plt.figure(figsize=(5,5))
ax = plt.subplot()

# set up the subplots as needed
ax.set_xlim((0, 10))
ax.set_ylim((0, 10))
ax.set_xlabel('x')
ax.set_ylabel('y')

# create objects that will change in the animation. These are
# initially empty, and will be given new values for each frame
# in the animation.
pts = []
N = 10 # Number of particles
for i in range(N):
    pts.append(ax.plot([], [], 'b.', ms=20)) # ax.plot returns a list of 2D line objects
    plt.close()

In [ ]:
def getAcc(pos):
    """
    Calculate the acceleration on each particle due to Newton's Law 
    pos  is an N x 3 matrix of positions
    mass is an N x 1 vector of masses
    G is Newton's Gravitational constant
    softening is the softening length
    a is N x 3 matrix of accelerations
    """
    
    N = pos.shape[0]
    a = np.zeros((N,2))
    softening = 0.001 # softening length
    G = 1.0         # Newton's Gravitational Constant
    global mass

    for i in range(N):
        for j in range(N):
            dx = pos[j,0] - pos[i,0];
            dy = pos[j,1] - pos[i,1];
            inv_r3 = (dx**2 + dy**2 + softening**2)**(-1.5);
            a[i,0] +=  G * (dx * inv_r3) * mass[j];
            a[i,1] +=  G * (dy * inv_r3) * mass[j];
            
    return a

# Simulation parameters
dt        = 0.01 # timestep

# Generate Initial Conditions
#np.random.seed(42) # set the random number generator seed
mass = 20.0*np.ones((N,1))/N  # total mass of particles is 20
pos  = np.random.normal(5,2,(N,2))#np.random.randn(N,2)+5 # randomly selected positions and velocities
vel  = np.random.normal(0,1,(N,2))#np.random.randn(N,2)

# calculate initial gravitational accelerations
acc = getAcc(pos)

def disegnaframe(t):
    global pos
    global vel
    global acc
    global dt
    
    # (1/2) kick
    vel += acc * dt/2.0

    # drift
    pos += vel * dt

    # update accelerations
    acc = getAcc(pos)

    # (1/2) kick
    vel += acc * dt/2.0

    for i in range(N):
        pts[i][0].set_data(pos[i,0], pos[i,1])

    ax.set_title('Tempo = {0:4d} s'.format(t))

In [ ]:
anim = animation.FuncAnimation(fig, disegnaframe, frames=100, interval=20)

In [ ]:
HTML(anim.to_html5_video())

### N punti, forza gravitazionale, merging

In [232]:
# create a figure and axes
fig = plt.figure(figsize=(5,5))
ax = plt.subplot()

N = 25 
plt.close()

In [233]:
def getAcc(pos):
    """
    Calculate the acceleration on each particle due to Newton's Law 
    pos  is an N x 3 matrix of positions
    mass is an N x 1 vector of masses
    G is Newton's Gravitational constant
    softening is the softening length
    a is N x 3 matrix of accelerations
    """
    global mass

    N = pos.shape[0]

    a = np.zeros((N,2))
    softening = 0.000001
    G = 1.0         # Newton's Gravitational Constant

    for i in range(N):
        for j in range(N):
            dx = pos[j,0] - pos[i,0]
            dy = pos[j,1] - pos[i,1]
            inv_r3 = (dx**2 + dy**2 + softening**2)**(-1.5)
            a[i,0] +=  G * (dx * inv_r3) * mass[j]
            a[i,1] +=  G * (dy * inv_r3) * mass[j]

    return a

def merge():
    global pos
    global vel
    global acc
    global mass
    
    N = pos.shape[0]
    soglia_r = 0.1
    
    merge_indices = []
    #merging
    for i in range(N):
        for j in np.arange(i+1,N,1):
            dx = pos[j,0] - pos[i,0]
            dy = pos[j,1] - pos[i,1]
            if((dx**2 + dy**2) < soglia_r):
                merge_indices.append(i)
                vel[j,:] = (mass[j]*vel[j,:] + mass[i]*vel[i,:])/(mass[j] + mass[i])
                mass[j] += mass[i] 

    pos  = np.delete(pos,merge_indices,axis=0)
    vel  = np.delete(vel,merge_indices,axis=0)
    acc  = np.delete(acc,merge_indices,axis=0)
    mass = np.delete(mass,merge_indices,axis=0)
    
# Simulation parameters
dt = 0.01 # timestep

# Generate Initial Conditions
#np.random.seed(42) # set the random number generator seed
mass = 20.0*np.ones((N,1))/N  # total mass of particles is 20
pos  = np.random.normal(5,2,(N,2))#np.random.randn(N,2)+5 # randomly selected positions and velocities
vel  = np.random.normal(0.1,1,(N,2))#np.random.randn(N,2)
acc  = np.zeros((N,2))#set acc to zeros

mass0 = mass[0] #starting mass of particles, needed for rescaling size of dots
merge()

# calculate initial gravitational accelerations
acc = getAcc(pos)

def disegnaframe(t):
    global pos
    global vel
    global acc
    global dt
    
    # (1/2) kick
    vel += acc * dt/2.0

    # drift
    pos += vel * dt
    
    #check for mergings
    merge()

    # update accelerations
    acc = getAcc(pos)

    # (1/2) kick
    vel += acc * dt/2.0

    N = pos.shape[0]
    pts = []
    ax.clear()
    for i in range(N):
        pts.append(ax.plot(pos[i,0], pos[i,1], 'b.', ms=20*(mass[i]/mass0)**0.5))
  
    # set up the subplots as needed
    ax.set_xlim((0, 10))
    ax.set_ylim((0, 10))
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title('Tempo = {0:4d} s'.format(t))

In [234]:
anim = animation.FuncAnimation(fig, disegnaframe, frames=200, interval=20)

In [235]:
HTML(anim.to_html5_video())

## <font color='red'>merging tra masse se r < soglia, distribuzione uniforme vs gaussiana, orbite aperte vs chiuse</font>  

# Da https://medium.com/swlh/create-your-own-n-body-simulation-with-python-f417234885e9